In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test_labels.csv', 'test.csv']


# Análisis de un problema multilabel

Competencia original:

https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

# EDA (Exploratory Data Analysis)

In [20]:
folder = '../input/'

In [3]:
train = pd.read_csv(folder+"train.csv")
test = pd.read_csv(folder+"test.csv")
test_labels = pd.read_csv(folder+"test_labels.csv")
submission = pd.read_csv(folder+"sample_submission.csv")

In [4]:
train.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
# Cantidad de observaciones
train.shape

(159571, 8)

## Defino salida del modelo $y$

In [6]:
# Defino y (Salida del modelo)
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
print(y.shape)
print(y[:10])

(159571, 6)
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 1 1 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [14]:
# Dataset muy desbalanceado
toxic_ratio = (y.sum(axis = 1) > 0).sum()/y.shape[0]
print('Porcentaje de comentarios toxicos:', toxic_ratio)

Porcentaje de comentarios toxicos: 0.10167887648758234


In [18]:
# La mayoría son toxic
print(train[list_classes].sum())
print()
print(train[list_classes].sum()/y.shape[0])

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64


In [19]:
# Superposición entre las clases (Multilabel)
for cl in list_classes[1:]:
    N = ((train['toxic'] == 0) & (train[cl] == 1)).sum()
    print(f'Es {cl} pero no es toxic:', N)

Es severe_toxic pero no es toxic: 0
Es obscene pero no es toxic: 523
Es threat pero no es toxic: 29
Es insult pero no es toxic: 533
Es identity_hate pero no es toxic: 103


Se puede ver aca por ejemplo que si es severe_toxic es si o si toxic, pero que insulto puede no ser tóxico por ejemplo

In [21]:
# Baseline (Suponer que siempre elijo zeros (No toxico))
1-(train[list_classes].sum().values/len(train)).mean()

0.9633412921729722

In [52]:
((y == np.zeros_like(y)).sum(axis=0)/len(y)).mean()

0.9633412921729722

In [22]:
train[list_classes].sum().values/len(train)

array([0.09584448, 0.00999555, 0.05294822, 0.00299553, 0.04936361,
       0.00880486])

# Divido entre train y valid

In [23]:
X_train, X_valid, Y_train, Y_valid = train_test_split(train['comment_text'], y, test_size = 0.1)

print(X_train.shape, X_valid.shape)
print(Y_train.shape, Y_valid.shape)

(143613,) (15958,)
(143613, 6) (15958, 6)


# Armo TFIDF

In [24]:
X_train[:10]

98458     "RfC: controversial multi-page move ==\n\n Sum...
39047     Wikify\nI have begun wikifying this article in...
7825      "\n Well, have a look at this photo, especiall...
9962      "\n\nThe message I left on your talk page begi...
131435    "\nAhh didn't notice that it was after the nex...
55735     Re \n\nAre you stalking me now? Don't you have...
126330    "\n\nHere's why there was no consensus looking...
15620            matt J! \n\nmatty j is a right winged nut!
56449     "\n\Go to my userpage! Just click on ""codelyo...
117705    Kangaroo, wallaroo and wallaby have the same p...
Name: comment_text, dtype: object

In [25]:
raw_text_train = X_train.apply(str.lower)
raw_text_valid = X_valid.apply(str.lower)
raw_text_test = test["comment_text"].apply(str.lower)

In [26]:
print(raw_text_train[:10]) # Recordar que train_test_split hace shuffle 

98458     "rfc: controversial multi-page move ==\n\n sum...
39047     wikify\ni have begun wikifying this article in...
7825      "\n well, have a look at this photo, especiall...
9962      "\n\nthe message i left on your talk page begi...
131435    "\nahh didn't notice that it was after the nex...
55735     re \n\nare you stalking me now? don't you have...
126330    "\n\nhere's why there was no consensus looking...
15620            matt j! \n\nmatty j is a right winged nut!
56449     "\n\go to my userpage! just click on ""codelyo...
117705    kangaroo, wallaroo and wallaby have the same p...
Name: comment_text, dtype: object


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

max_features = 10000

tfidf_vectorizer = TfidfVectorizer(max_df=0.11, min_df=1,
                                   max_features=max_features,
                                   stop_words='english')

%time tfidf_matrix_train = tfidf_vectorizer.fit_transform(raw_text_train)

CPU times: user 8.87 s, sys: 88 ms, total: 8.96 s
Wall time: 8.98 s


In [28]:
%time tfidf_matrix_valid = tfidf_vectorizer.transform(raw_text_valid)

CPU times: user 956 ms, sys: 0 ns, total: 956 ms
Wall time: 955 ms


In [29]:
count_vectorizer = CountVectorizer(max_df=0.11, min_df=1,
                                   max_features=max_features,
                                   stop_words='english')

%time count_matrix_train = count_vectorizer.fit_transform(raw_text_train)

CPU times: user 8.83 s, sys: 60 ms, total: 8.89 s
Wall time: 8.9 s


In [30]:
tfidf_matrix_train.shape, count_matrix_train.shape

((143613, 10000), (143613, 10000))

# Sparsity

In [31]:
sparsity = 1 - (tfidf_matrix_train>0).sum()/(tfidf_matrix_train.shape[0]*tfidf_matrix_train.shape[1])
print(sparsity)

0.9979016850842194


# Resultados TFIDF

In [32]:
top_10 = np.argsort(tfidf_matrix_train.sum(axis=0))[0,::-1][0,:10].tolist()[0]
feature_names = np.array(tfidf_vectorizer.get_feature_names())
print(feature_names[np.array(top_10)])

['thanks' 'edit' 'know' 'think' 'did' 'user' 'people' 'articles' 'time'
 'good']


In [33]:
top_10_count = np.argsort(count_matrix_train.sum(axis=0))[0,::-1][0,:10].tolist()[0]
feature_names_count = np.array(count_vectorizer.get_feature_names())
print(feature_names_count[np.array(top_10_count)])

['think' 'know' 'edit' 'people' 'articles' 'use' 'time' 'did' 'user'
 'thanks']


In [34]:
dense_matrix_train = tfidf_matrix_train.todense()
dense_matrix_valid = tfidf_matrix_valid.todense()

# Creación del modelo

In [35]:
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers

Using TensorFlow backend.


In [36]:
input_features = dense_matrix_train.shape[1]
output_size = Y_train.shape[1]

model_rl = Sequential()
model_rl.add(Dense(output_size, input_dim=input_features, activation='sigmoid', 
                   kernel_initializer=initializers.normal(mean=0, stddev=0.001)))
model_rl.summary()
model_rl.compile('Adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 60006     
Total params: 60,006
Trainable params: 60,006
Non-trainable params: 0
_________________________________________________________________


In [37]:
model_rl.evaluate(dense_matrix_valid, Y_valid)

15958/15958 [==============================] - 4s 278us/step


[0.6931596473204878, 0.48956637050307983]

In [38]:
batch_size = 128
epochs = 20
model_rl.fit(dense_matrix_train, 
          Y_train, 
          batch_size = batch_size,
          epochs=epochs, 
          verbose=1, 
          validation_data=(dense_matrix_valid, Y_valid))

Instructions for updating:
Use tf.cast instead.
Train on 143613 samples, validate on 15958 samples
Epoch 1/20
143613/143613 [==============================] - 10s 68us/step - loss: 0.3332 - acc: 0.9633 - val_loss: 0.1830 - val_acc: 0.9651
Epoch 2/20
143613/143613 [==============================] - 8s 55us/step - loss: 0.1466 - acc: 0.9655 - val_loss: 0.1221 - val_acc: 0.9671
Epoch 3/20
143613/143613 [==============================] - 8s 56us/step - loss: 0.1103 - acc: 0.9679 - val_loss: 0.1009 - val_acc: 0.9692
Epoch 4/20
143613/143613 [==============================] - 8s 55us/step - loss: 0.0944 - acc: 0.9703 - val_loss: 0.0896 - val_acc: 0.9714
Epoch 5/20
143613/143613 [==============================] - 8s 55us/step - loss: 0.0847 - acc: 0.9725 - val_loss: 0.0824 - val_acc: 0.9733
Epoch 6/20
143613/143613 [==============================] - 8s 56us/step - loss: 0.0780 - acc: 0.9742 - val_loss: 0.0774 - val_acc: 0.9745
Epoch 7/20
143613/143613 [==============================] - 8s 55u

# Interpretación

In [39]:
(model_rl.get_weights()[0]).shape

(10000, 6)

- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

In [40]:
salida = 3
sorted_indexes = np.argsort(model_rl.get_weights()[0][:,salida])[::-1]
np.array(tfidf_vectorizer.get_feature_names())[sorted_indexes][:20]

array(['kill', 'die', 'rape', 'burn', 'shoot', 'death', 'hope', 'fucking',
       'going', 'ass', 'punch', 'kick', 'destroy', 'house', 'deserve',
       'hunt', 'raped', 'blood', 'beat', 'live'], dtype='<U39')

# MLP

In [41]:
from keras import regularizers
from keras import initializers
from keras.layers import Activation
from keras import optimizers

In [42]:
default_initializer = initializers.normal(mean=0, stddev=0.01)


In [43]:
input_features = dense_matrix_train.shape[1]
output_size = Y_train.shape[1]
hidden_units = 100
lambd = 0 #0.001
model_sig_nn = Sequential()
model_sig_nn.add(Dense(200,
                       input_dim=input_features, 
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Oculta_1"))
model_sig_nn.add(Activation('sigmoid'))
model_sig_nn.add(Dense(200,
                       input_dim=input_features, 
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Oculta_2"))
model_sig_nn.add(Activation('sigmoid'))
model_sig_nn.add(Dense(output_size,
                       kernel_regularizer=regularizers.l2(lambd), 
                       kernel_initializer=default_initializer,
                       name="Capa_Salida"))
model_sig_nn.add(Activation('sigmoid', name="output")) 
model_sig_nn.summary()


lr = 0.001 
batch_size = 256
epochs = 10

#selectedOptimizer = optimizers.SGD(lr=lr)
selectedOptimizer = optimizers.adam(lr=lr, decay=0.001)

model_sig_nn.compile(loss = 'binary_crossentropy', optimizer=selectedOptimizer, 
                     metrics=['accuracy']) #auc

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Capa_Oculta_1 (Dense)        (None, 200)               2000200   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
Capa_Oculta_2 (Dense)        (None, 200)               40200     
_________________________________________________________________
activation_2 (Activation)    (None, 200)               0         
_________________________________________________________________
Capa_Salida (Dense)          (None, 6)                 1206      
_________________________________________________________________
output (Activation)          (None, 6)                 0         
Total params: 2,041,606
Trainable params: 2,041,606
Non-trainable params: 0
_________________________________________________________________


In [44]:
model_sig_nn.evaluate(dense_matrix_valid, Y_valid)

15958/15958 [==============================] - 1s 83us/step


[0.6936332652825493, 0.49924803879763313]

In [45]:
history = model_sig_nn.fit(dense_matrix_train, 
          Y_train, 
          batch_size = batch_size,
          epochs=epochs, 
          verbose=1, 
          validation_data=(dense_matrix_valid, Y_valid), 
         )

Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143613/143613 [==============================] - 9s 66us/step - loss: 0.1480 - acc: 0.9621 - val_loss: 0.1351 - val_acc: 0.9639
Epoch 2/10
 83968/143613 [================>.............] - ETA: 3s - loss: 0.1352 - acc: 0.9634

KeyboardInterrupt: 

In [ ]:
pred_valid = model_sig_nn.predict(dense_matrix_valid, verbose = 1)
pred_train = model_sig_nn.predict(dense_matrix_train, verbose = 1)

# Curva ROC

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

print(roc_auc_score(Y_train, pred_train, average='macro'))
print(roc_auc_score(Y_valid, pred_valid, average='macro'))

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = Y_valid.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_valid[:, i], pred_valid[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
fpr["micro"], tpr["micro"], _ = roc_curve(Y_valid.ravel(), pred_valid.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
from matplotlib import pyplot as plt
# Compute macro-average ROC curve and ROC area
lw = 2
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()